In [ ]:
%load_ext supriya.ext.ipython
import supriya

## Realtime Server Node Tree Model

In [ ]:
synth_a = supriya.Synth(frequency=333, name="A")
synth_b = supriya.Synth(frequency=444, name="B")
synth_c = supriya.Synth(frequency=555, name="C")
outer_group = supriya.Group(name="Outer")
inner_group = supriya.Group(name="Inner")
outer_group.extend([synth_a, inner_group, synth_c])
inner_group.append(synth_b)

In [ ]:
_ = supriya.graph(outer_group)

In [ ]:
# groups know about their children
for index, node in enumerate(outer_group.children):
    print(index, repr(node)) 

In [ ]:
# and nodes know about their parents
for distance, node in enumerate(synth_b.parentage):
    print(distance, repr(node))

In [ ]:
# nodes can be iterated depthfirst and know their position in the tree
for child in outer_group.depth_first():
    print(repr(child), child.graph_order)

In [ ]:
# booting is just like in sclang
server = supriya.Server()
server.boot()

In [ ]:
# printing asks for the string representation of an object
# string representation are/can be different from interpreter representations
# we can query the node tree via /g_queryTree requests
print(server)

In [ ]:
# allocating a group also recursively allocates its children
outer_group.allocate()
print(server)

In [ ]:
# we can visualize the allocated node structure
_ = supriya.graph(outer_group)

In [ ]:
# TODO: synth and group control interfaces, and their mutability
synth_a["frequency"]

In [ ]:
# we can set synth controls, treating the synth like a dictionay
synth_a["frequency"] = 666
# and query them
print(synth_a["frequency"])

In [ ]:
# the synth controls are also explicitly modeled, hidden inside the "controls" interface
for control_name in synth_a.controls:
    print(repr(synth_a.controls[control_name]))

In [ ]:
# groups also have a control interface, aggregating controls from synths in their subtree
for control_name in outer_group.controls:
    print(repr(outer_group.controls[control_name]))

In [ ]:
# we can allocate new nodes and move existing nodes in the same command
# synthdefs are also automatically allocated
# we use the completion message of the /s_new to perform the node allocation / movement
# this synth uses a non-default synthdef, which hasn't previously been allocated on the server
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.pad)
inner_group.extend([synth_d, synth_a])

In [ ]:
# iteration continues to work
for node in outer_group.children:
    print(repr(node))

In [ ]:
for control_name in outer_group.controls:
    print(control_name)

In [ ]:
# we can also visualize the entire server node structure, including the root node and default group
_ = supriya.graph(server)

In [ ]:
outer_group.free()

In [ ]:
print(server)

In [ ]:
# explicitly freeing a group does not destructure its children
print(outer_group)

In [ ]:
_ = supriya.graph(outer_group)

In [ ]:
server.quit()

## OSC Command Aggregation

In [ ]:
server = supriya.Server().boot()
server.debug_request_names = True

In [ ]:
synth_a = supriya.Synth(frequency=333)
synth_b = supriya.Synth(frequency=444)
synth_c = supriya.Synth(frequency=555)
outer_group = supriya.Group()
inner_group = supriya.Group()
outer_group.extend([synth_a, inner_group, synth_c])
inner_group.append(synth_b)
print(outer_group)

In [ ]:
# let's allocate the default synthdef manually (you'll see why soon)
supriya.assets.synthdefs.default.allocate()

In [ ]:
# we can spy on osc messages going to and coming from scsynth
with server.osc_io.capture() as transcript:
    outer_group.allocate()

In [ ]:
# what was sent when we allocated that group? an osc bundle
# supriya models osc bundles and osc messages explicitly as classes
# the osc messages are a linearized version of depth-first allocation of the nodes in the subtree
for timestamp, osc_message in transcript.sent_messages:
    print(repr(osc_message))

In [ ]:
# we also have the responses from the server to each of those /s_new and /g_new commands
# and the /synced response as well
for timestamp, osc_message in transcript.received_messages:
    print(repr(osc_message))

In [ ]:
# recall that i manually allocated the default synthdef earlier
# let's make a new synth_d using a simple sine-wave synthdef
# let's allocate the new synth and also move synth 1001 into the inner group
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.simple_sine)
with server.osc_io.capture() as transcript:
    inner_group.extend([synth_d, synth_a])

In [ ]:
# supriya knows if synthdefs have previously been allocated
# when allocating new synths it will generate an /d_recv and
# add any node allocation / movement / free commands as the completion message
for timestamp, osc_message in transcript.sent_messages:
    print(repr(osc_message))

In [ ]:
for timestamp, osc_message in transcript.received_messages:
    print(repr(osc_message))

In [ ]:
server.quit()

## Requests and Responses

In [ ]:
server = supriya.Server().boot()

In [ ]:
# ok, this is almost the same as before, just simpler
synth_a = supriya.Synth(frequency=333)
synth_b = supriya.Synth(synthdef=supriya.assets.synthdefs.pad, frequency=444)
outer_group = supriya.Group()
inner_group = supriya.Group()
outer_group.extend([synth_a, inner_group])
inner_group.append(synth_b)

In [ ]:
with server.osc_io.capture() as transcript:
    outer_group.allocate()

In [ ]:
for timestamp, request in transcript.requests:
    print(request)

None of the above is OSC. It's all explicitly class-modeled.

Note that some of the `node_id` and `target_node_id` arguments are actually references to specific `Group` or `Synth` objects rather than just integers.

When communicating a request like this to the server, we don't necessarily know the IDs of the nodes until we start to communicate it.

- Linearize the request (if necessary) into a series of requests.
- Apply each request _locally_, including allocating the ID of each request's node; the request classes implement any necessary logic for local application.
- If we want to block until the server processes the request, register an OSC callback using the requests's knowledge of what to expect
- Convert the request to OSC and send it
- If blocking, wait until we receive the expected response.

In [ ]:
for timestamp, response in transcript.responses:
    print(response)

In [ ]:
print(dir(supriya.commands))

The synthesis server is a state machine.

OSC commands are state transitions.

The local server is a lossy model of the (complete) synthesis server state.

## SynthDef Builders

SynthDefs are built via context managers, not via the namespaces of functions

SynthDefs do not need to be named; Supriya uses hashing to generate unique names

## SynthDef Factories

## Non-realtime Session Model

## The `__render__` protocol

## NRT Dependency Tree (turtles all the way down)

## (N)RT Patterns

## CLI Tooling and NRT Projects